In [ ]:
!pip install qiskit
!pip install qiskit-machine-learning

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.neural_networks import EstimatorQNN


In [ ]:
n_qubits = 4

In [ ]:
def layer(W):
    """Applies a layer of arbitrary rotations and circular entanglements to the variational circuit

    Args:
        W (np.ndarray): rotation parameters for the layer
    """
    qc = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        qc.u(W[i*3], W[i*3 + 1], W[i*3 + 2], i)

    for i in range(n_qubits-1):
        qc.cnot(i, i+1)

    if n_qubits > 2:
        qc.cnot(n_qubits-1, 0)

    return qc

In [ ]:
def statepreparation(x):
    """Prepares the binary state fed to the vqc

    Args:
        x (List): list of 0s and 1s corresponding to the basis state
    """
    qc = QuantumCircuit(n_qubits)
    
    for i, x_i in enumerate(x):
        qc.x(i) if x_i == 1 else qc.id(i)

    return qc

In [ ]:
n_layers = 2

weight_params = ParameterVector(3 * n_qubits * n_layers, name='W')
input_params = ParameterVector(n_qubits, name='x')

In [ ]:
# prepare the quantum circuit
qc = QuantumCircuit(n_qubits)
qc.compose(statepreparation(input_params))
for l in range(n_layers):
    qc.compose(layer(weight_params[3 * n_qubits * l: 3 * n_qubits * (l+1)]))


In [ ]:
# define an observable
observable = SparsePauliOp.from_list([("ZIII", 1)])

In [ ]:
qnn = EstimatorQNN(circuit=qc, observables= observable, input_params=input_params, weight_params=weight_params)